In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../flaming-choripan') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [2]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms

kf = '.'
lcset_name = 'train'
cfilename = f'survey=alerceZTFv7.1~bands=gr~mode=onlySNe'
methods = [
    'linear-fstw',
    'bspline-fstw',
    'spm-mle-fstw',
    'spm-mle-estw',
    'spm-mcmc-fstw',
    'spm-mcmc-estw',
    ]

In [3]:
for method in methods:
    any_incorrects_fittings = sms.get_any_incorrects_fittings(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(f'method={method} - any_incorrects_fittings={len(any_incorrects_fittings)} #')
    
for method in methods:
    all_incorrects_fittings = sms.get_all_incorrects_fittings(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(f'method={method} - all_incorrects_fittings={len(all_incorrects_fittings)} #')

method=linear-fstw - any_incorrects_fittings=111 #
method=bspline-fstw - any_incorrects_fittings=296 #
method=spm-mle-fstw - any_incorrects_fittings=111 #
method=spm-mle-estw - any_incorrects_fittings=111 #
method=spm-mcmc-fstw - any_incorrects_fittings=133 #
method=spm-mcmc-estw - any_incorrects_fittings=75 #
method=linear-fstw - all_incorrects_fittings=1 #
method=bspline-fstw - all_incorrects_fittings=81 #
method=spm-mle-fstw - all_incorrects_fittings=1 #
method=spm-mle-estw - all_incorrects_fittings=1 #
method=spm-mcmc-fstw - all_incorrects_fittings=4 #
method=spm-mcmc-estw - all_incorrects_fittings=2 #


In [4]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms

for method in methods:
    xe_ptimes = sms.get_perf_times(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(f'method={method} - xe_ptimes={xe_ptimes}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
method=linear-fstw - xe_ptimes=.862±.344
method=bspline-fstw - xe_ptimes=.714±.235
method=spm-mle-fstw - xe_ptimes=10.023±3.823
method=spm-mle-estw - xe_ptimes=10.229±3.933
method=spm-mcmc-fstw - xe_ptimes=64.681±6.406
method=spm-mcmc-estw - xe_ptimes=66.613±7.866


In [5]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms
from flamingchoripan.latex.latex_tables import LatexTable

info_df = sms.get_info_dict(f'../save/ssne', methods, cfilename, kf, lcset_name)
latex_kwargs = {
    'caption':f'kf={kf}',
    'label':'???',
    #'bold_criteriums':{c:'max' if ('%' in c or 'fits-n' in c) else 'min' for c in info_df.columns},
    #'custom_tabular_align':'l|'+'c'*sum([m.split('-')[-1]=='fstw' for m in methods])+'|'+'c'*sum([m.split('-')[-1]=='estw' for m in methods]),
    #'custom_tabular_align':'l|c|ccc|ccc|ccc',
}
latex_table = LatexTable(info_df, **latex_kwargs)
print(latex_table)
info_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\def\srule{\rule{0pt}{2ex}\rule[-1ex]{0pt}{0ex}}
\begin{table*}
\centering
\caption{kf=.}
\label{???}\vspace{.1cm}
\begin{tabular}{lc|cccccc}
\hline
metric & band & linear-fstw & bspline-fstw & spm-mle-fstw & spm-mle-estw & spm-mcmc-fstw & spm-mcmc-estw \srule\\\hline\hline
trace-time [segs] & . & .820$\pm$.355 & .648$\pm$.248 & 9.594$\pm$3.938 & 9.798$\pm$4.045 & 61.290$\pm$11.562 & 62.198$\pm$13.495  \srule\\
fit-log-error & g & 1.640$\pm$.523 & 4.203$\pm$2.205 & 2.790$\pm$1.299 & 2.791$\pm$1.296 & 3.409$\pm$.855 & 3.392$\pm$.852  \srule\\
fits-success [\%] & g & 92.784 & 78.938 & 92.784 & 92.784 & 91.667 & 89.687  \srule\\
fit-log-error & r & 1.749$\pm$.461 & 4.944$\pm$2.045 & 3.428$\pm$1.233 & 3.429$\pm$1.236 & 3.744$\pm$.858 & 3.721$\pm$.883  \srule\\
fits-success [\%] & r & 97.595 & 88.185 & 97

,linear-fstw,bspline-fstw,spm-mle-fstw,spm-mle-estw,spm-mcmc-fstw,spm-mcmc-estw
metric=trace-time [segs]~band=.,.820±.355,.648±.248,9.594±3.938,9.798±4.045,61.290±11.562,62.198±13.495
metric=fit-log-error~band=g,1.640±.523,4.203±2.205,2.790±1.299,2.791±1.296,3.409±.855,3.392±.852
metric=fits-success [%]~band=g,92.783505,78.938466,92.783505,92.783505,91.666667,89.686924
metric=fit-log-error~band=r,1.749±.461,4.944±2.045,3.428±1.233,3.429±1.236,3.744±.858,3.721±.883
metric=fits-success [%]~band=r,97.594502,88.184601,97.594502,97.594502,96.563574,96.132597


In [6]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms

for method in methods:
    rank_bdict = sms.get_ranks(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(method)
    for b in rank_bdict.keys():
        print(rank_bdict[b])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
linear-fstw
band=g:
[1#] 0@ZTF18aasdted=31.8127
[2#] 0@ZTF18aaslhxt=14.7634
[3#] 0@ZTF19aatlmbo=14.5121
[4#] 0@ZTF19aawgopm=13.8231
[5#] 0@ZTF18abuqugw=13.6038
[6#] 0@ZTF18aayjyub=13.5225
[7#] 0@ZTF18aaxcntm=13.3264
[8#] 0@ZTF18aaqkcso=13.1993
[9#] 0@ZTF19aagmsrr=13.0850
[10#] 0@ZTF18acqqyah=13.0708
band=r:
[1#] 0@ZTF18aasdted=32.7411
[2#] 0@ZTF19aatlmbo=18.2164
[3#] 0@ZTF18aaqkcso=15.7637
[4#] 0@ZTF19aawgopm=15.0970
[5#] 0@ZTF18aaxcntm=14.8191
[6#] 0@ZTF19aacgslb=14.4614
[7#] 0@ZTF18aaumeys=14.3597
[8#] 0@ZTF18acqqyah=14.0312
[9#] 0@ZTF18acrdlrp=13.8756
[10#] 0@ZTF18aahvndq=13.6651
bspline-fstw
band=g:
[1#] 0@ZTF18abuhzfc=9,746.3597
[2#] 0@ZTF19aawmkja=9,718.2961
[3#] 0@ZTF18abwkrbl=9,617.5396
[4#] 0@ZTF19aakljys=9,522.8373
[5#] 0@ZTF19aanonjt=9,521.0885
[6#] 0@ZTF19aanuhox=9,281.5141
[7#] 0@ZTF19aalzmmt=9,164.5237
[8#] 0@ZTF19abjibet=8,988.6626
[9#] 0@ZTF19aavhypb=8,984.4749
[10#] 0@ZTF18acsyanq=8